# SOAP_similarity_kernal

In [ ]:
from dscribe.descriptors import SOAP
species = ["Ge", "Te"]
r_cut = 3.0
sigma = 0.3
n_max = 8
l_max = 8
soap = SOAP(species=species, r_cut=r_cut, sigma=sigma, n_max=n_max, l_max=l_max)
soap.get_number_of_features()

In [ ]:
from ase.io import read
import numpy as np

xyz_files = read("../NEP_Order_parameter/split_movie_files/split_100.xyz", index=':')
soap_vectors = []
for atoms in xyz_files:
    Ge_indices = [atom.index for atom in atoms if atom.symbol == 'Ge']
    soap_vector = soap.create(atoms, centers=Ge_indices)
    soap_vector_avg = np.mean(soap_vector, axis=0)
    soap_vectors.append(soap_vector_avg)

p_j = np.mean(soap_vectors, axis=0).reshape(1, -1)
p_j_norm = p_j / np.linalg.norm(p_j)

In [ ]:
from ase.io import read
import matplotlib.pyplot as plt
import numpy as np
import scienceplots
plt.style.use(['science','ieee','no-latex','bright'])

def _plot(ax, filename, p_j_norm, label):
    xyz_files = read(filename, index='::100')
    kj_list = []
    for atoms in xyz_files:
        Ge_indices = [atom.index for atom in atoms if atom.symbol == 'Ge']
        for i in Ge_indices:
            p_i = soap.create(atoms, centers=[i])
            p_i_norm = p_i / np.linalg.norm(p_i)
            k_j = np.dot(p_i_norm, p_j_norm.T) ** 2
            kj_list.append(k_j[0][0])

    bins = np.linspace(0, 1, 1001)
    hist, bin_edges = np.histogram(kj_list, bins=bins)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    plt.plot(bin_centers, hist, linestyle='-', label=label)

plt.figure(figsize=(4, 3))
ax1 = plt.gca()
_plot(ax1, "../NEP_Order_parameter/split_movie_files/split_300.xyz", p_j_norm, label="300K")
_plot(ax1, "../NEP_Order_parameter/split_movie_files/split_800.xyz", p_j_norm, label="800K")
plt.xlabel(r'$k_j$ Value')
plt.ylabel('Distribution')
plt.xlim(0.8, 1)
plt.ylim(0, None)
plt.yticks([])
plt.legend()
plt.show()